In [1]:
import time
import os
import json
import tensorflow as tf
from tensorflow.keras import utils
from tensorflow.keras import backend as K
from tensorflow.keras.callbacks import EarlyStopping, ModelCheckpoint, TensorBoard, ReduceLROnPlateau, LearningRateScheduler
from tensorflow.keras.models import Model, load_model, Sequential
from tensorflow.keras.layers import *
from tensorflow.keras.preprocessing import image

#from keras.preprocessing import image
from keras.applications.imagenet_utils import preprocess_input
from matplotlib.pyplot import imread, imshow

#from keras.preprocessing.image import ImageDataGenerator
#from keras.layers import Dense, Dropout, Activation, Flatten
#from keras.layers import Conv2D, MaxPooling2D

import numpy as np
import matplotlib.pyplot as plt
import random
from datetime import datetime
import nibabel as nib
import re
from collections import Counter
#import imageio
from nst_utils import *

%matplotlib inline
%load_ext autoreload
%autoreload 1

#%aimport 

SEED=1
random.seed(SEED)
np.random.seed(SEED)
tf.set_random_seed(SEED)

K.clear_session()
#K.set_image_data_format('channels_last')
#K.set_learning_phase(1)



Using TensorFlow backend.


In [2]:
print(tf.__version__)

#with tf.Session() as ts:
#    vmodel = load_vgg_model("/home/ubuntu/fmriNet/fmriNet/pretrainedModel/imagenet-vgg-verydeep-19.mat")
#    print(vmodel)



1.14.0


# Load data

For phase1, training examples are images shown to 4 participants across multiple sessions.

Images labeled for 3 classes: scenes, coco, imgnet


In [3]:
stList = {}
stimulusDirPath = os.path.join('images', 'BOLD5000_Stimuli', 'Scene_Stimuli', 'Presented_Stimuli')
print("stimulusDirPath: %s" % stimulusDirPath)
# "CSI1", "CSI2", "CSI3"
#"start_sess": 1,
#        "last_sess": 14,
#        "start_run": 1,
#        "last_run": 10

#"start_sess": 14,
#        "last_sess": 15,
#        "start_run": 1,
#        "last_run": 10
            
data_split = {
    "train": {
        "participant_list": ["CSI1"],
        "start_sess": 1,
        "last_sess": 3,
        "start_run": 1,
        "last_run": 4
    },
    "dev": {
        "participant_list": ["CSI1"],
        "start_sess": 14,
        "last_sess": 15,
        "start_run": 1,
        "last_run": 2
    }
}
classes = {'ImageNet': 0, 'COCO': 1, 'Scene': 2}

# Get list of stimuli pictures shown in each session in each run
for data_type, items in data_split.items():
    stList[data_type] = {}
    for participant in items['participant_list']:
        
        # CS1 file are missing 1 after CSI
        if participant == "CSI1":
            CSI = "CSI"
        else:
            CSI = participant
        
        stList[data_type][participant] = {}
        for sNum in range(items['start_sess'], items['last_sess']):
            sSes = "sess" + str(sNum).zfill(2)
            stList[data_type][participant][sSes] = {}
            for rNum in range(items['start_run'], items['last_run']):
                sRun = "run" + str(rNum).zfill(2)
                dir_path = os.path.join("images","BOLD5000_Stimuli", "Stimuli_Presentation_Lists",participant, participant + "_" + sSes)
                #print(stimulusDirPath)
                stimulusListFilename = os.path.join(dir_path, "_".join([CSI, sSes, sRun]) + ".txt")
                #print(stimulusListFilename)
                with open(stimulusListFilename) as f:
                    stList[data_type][participant][sSes][sRun] = f.read().splitlines() 

            
x_images_path = {}
y_labels = {}
for data_type, participantDict in stList.items():
    x_images_path[data_type] = {}
    y_labels[data_type] = {}
    for participant, sessDict in participantDict.items(): 
        x_images_path[data_type][participant] = {}
        y_labels[data_type][participant] = {}
        for sess, runDict in sessDict.items():
            x_images_path[data_type][participant][sess] = {}
            y_labels[data_type][participant][sess] = {}
            for run, imageList in runDict.items():
                x_images_path[data_type][participant][sess][run] = []
                y_labels[data_type][participant][sess][run] = []
                #print("sess: %s, run: %s" %(sess, run))
                labelList = []
                for imageFileName in imageList:
                    for (currDir, _, fileList) in os.walk(stimulusDirPath):
                        currBaseDir = os.path.basename(currDir)
                        for filename in fileList:
                            if filename in imageFileName:
                                fullFilename = os.path.join(currDir, filename)
                                x_images_path[data_type][participant][sess][run].append(fullFilename)
                                # using directory path to determine class
                                labelList.append(classes.get(currDir.split('/')[-1]))
                                break
        
                y_labels[data_type][participant][sess][run] = np.reshape(np.asarray(labelList), (1, -1))

# Todo: normalize data
# x_train / 255.0, x_val/255.0, x_train/255.0

#print(x_images_path)
#print(y_labels["train"]["CSI1"]['sess01']['run01'].shape)
#print(y_labels["dev"]["CSI3"]['sess01']['run01'].shape)
#print(len(x_images_path["train"]["CSI1"]['sess01']['run02']))
print('done')

stimulusDirPath: images/BOLD5000_Stimuli/Scene_Stimuli/Presented_Stimuli
done


# Preprocess 
Compute feature vectors using pretrained imagenet-vgg-verydeep model

Feature vectors saved in file



In [ ]:
content_layer = 'avgpool5'
stimuli_features_dir = 'stimulifeatures'
def unrollContentOutput(cOutput):
    m, n_H, n_W, n_C = cOutput.shape
    output = np.transpose(np.reshape(cOutput, (n_H * n_W, n_C)))
    return output

In [ ]:
print("start time: %s" % datetime.now().strftime('%Y-%m-%dT%H:%M:%S'))


!mkdir -p stimulifeatures

tf.reset_default_graph()
#sess = tf.InteractiveSession()
#precompute content vectors from presented stimuli
#content_layer = 'conv4_2'
with tf.Session() as ts:
    vmodel = load_vgg_model("imagenet-vgg-verydeep-19.mat")
    for data_type, participantDict in x_images_path.items():
        for participant, sessDict in participantDict.items():
            for sess, runDict in sessDict.items():
                for run, imageList in runDict.items():
                    #x_content = {sess: {run: []}}
                    file_path= os.path.join(stimuli_features_dir, "_".join([participant, sess, run]) + ".npy")
                    if os.path.exists(file_path):
                        #print already computed, skip
                        continue

                    print("file_path: %s" % file_path)
                    print("participant: %s, sess: %s, run: %s" % (participant, sess, run))
                    contentList = []
                    for img_path in imageList:
                        #stImage = imread(cImage)
                        img = image.load_img(img_path, target_size=(375, 375))
                        x = image.img_to_array(img)
                        x = np.expand_dims(x, axis=0)
                        x = preprocess_input(x)
                        #print("img_path: %s" % img_path)
                        #print('Input image shape:', x.shape)
                        #img_array = img_to_array(img)
                        #stImage = imageio.imread(img_path)
                        #print("img_path: %s" % img_path)
                        #print(stImage.shape)
                        #stImage = reshape_and_normalize_image(stImage)
                        #stImage = np.reshape(stImage, (1, 375, 375, 3))
                        ts.run(vmodel['input'].assign(x))
                        #a_C = sess.run(vmodel)
                        out = vmodel[content_layer]
                        contentOut = ts.run(out)
                        contentList.append(unrollContentOutput(contentOut))
            
                    #x_content[sess][run] = np.asarray(contentList)
                    contentArray = np.asarray(contentList)
                    # shape is (35, 512, 144): num of pictures, channels, width*height
                    #print(x_content[sess][run].shape)
                    #x_content[sess][run].append(unrollContentOutput(contentOut))
        
                    #np.save(file_path, x_content)
                    np.save(file_path, contentArray)
                    #del x_content

print('done')
print("end time: %s" % datetime.now().strftime('%Y-%m-%dT%H:%M:%S'))

In [ ]:
with tf.Session() as ts:
    vmodel = load_vgg_model("imagenet-vgg-verydeep-19.mat")
    img_path = './images/BOLD5000_Stimuli/Scene_Stimuli/Presented_Stimuli/ImageNet/n01833805_1411.JPEG'
    img = image.load_img(img_path, target_size=(375, 375))
    x = image.img_to_array(img)
    x = np.expand_dims(x, axis=0)
    x = preprocess_input(x)
    ts.run(vmodel['input'].assign(x))
    out = vmodel[content_layer]
    predictContentOut = ts.run(out)

# Model

In [ ]:
num_classes = 3
VERSION = datetime.now().strftime('%Y-%m-%dT%H:%M:%S')
file_path = os.path.join('stimulifeatures', 'CSI2_sess01_run01.npy')

x_content = np.load(file_path, allow_pickle=True)
print(x_content.shape)

def dnn_classifier(input_shape, num_classes):
    X_input = Input(input_shape)
    X = Flatten()(X_input)
    X = Dense(64, activation='tanh')(X)
    #X = Dense(16, activation='tanh')(X)
    X = Dropout(0.2)(X)
    X = Dense(num_classes, activation='softmax')(X)
    model = Model(inputs = X_input, outputs = X, name='dnn_classifier')
    return model

def dnn_gap_classifier(input_shape, num_classes):
    X_input = Input(input_shape)
    X = GlobalAveragePooling1D(data_format='channels_first')(X_input)
    X = Dense(64, activation='relu')(X)
    X = Dropout(0.2)(X)
    #X = Activation('relu')(X)
    #X = Dropout(0.2)(X)
    X = Dense(num_classes, activation='softmax')(X)
    model = Model(inputs = X_input, outputs = X, name='dnn_classifier')
    return model


def auto_encoder(input_shape, encoding_dim):
    X_input = Input(input_shape)
    X = Dense(encoding_dim, activation='relu')(X_input)
    model = Model(inputs = X_input, outputs = X, name='dnn_classifier')
    return model

def cnn_classifier(input_shape, num_classes):
    X_input = Input(input_shape)
    X = Conv2D(32, (3, 3), padding='same')(X_input)
    X = Activation('relu')(X)
    X = Conv2D(32, (3, 3))(X)
    X = Activation('relu')(X)
    X = MaxPooling2D(pool_size=(2, 2))(X)
    X = Dropout(0.25)(X)
    X = Conv2D(64, (3, 3), padding='same')(X)
    X = Activation('relu')(X)
    X = Conv2D(64, (3, 3))(X)
    X = Activation('relu')(X)
    X = MaxPooling2D(pool_size=(2, 2))(X)
    X = Dropout(0.25)(X)
    X = Activation('relu')(X)
    X = MaxPooling2D(pool_size=(2, 2))(X)
    X = Dropout(0.25)(X)
    X = Flatten()(X)
    X = Dense(512)(X)
    X = Activation('relu')(X)
    X = Dense(num_classes)(X)
    X = Activation('softmax')(X)
    model = Model(inputs = X_input, outputs = X, name='cnn_classifier')
    return model

#model = tf.keras.models.Sequential([
#    tf.keras.layers.Flatten(input_shape=[512, 144]),
#    tf.keras.layers.Dense(128, activation='relu'),
#    tf.keras.layers.Dropout(0.2),
#    tf.keras.layers.Dense(num_classes, activation='softmax')
#])

#model = tf.keras.models.Sequential([
#    tf.keras.layers.Conv2D(32, (3, 3), padding='same',
#                 input_shape=x_train.shape[1:]),
#    tf.keras.layers.Activation('relu'),
#    tf.keras.layers.Conv2D(32, (3, 3)),
#    tf.keras.layers.Activation('relu'),
#    tf.keras.layers.MaxPooling2D(pool_size=(2, 2)),
#    tf.keras.layers.Dropout(0.25),
#    tf.keras.layers.Conv2D(64, (3, 3), padding='same'),
#    tf.keras.layers.Activation('relu'),
#    tf.keras.layers.Conv2D(64, (3, 3)),
#    tf.keras.layers.Activation('relu'),
#    tf.keras.layers.MaxPooling2D(pool_size=(2, 2)),
#    tf.keras.layers.Dropout(0.25),
#    tf.keras.layers.Flatten(),
#    tf.keras.layers.Dense(512),
#    tf.keras.layers.Activation('relu'),
#    tf.keras.layers.Dense(num_classes),
#    tf.keras.layers.Activation('softmax')
#])


#input_shape=[512, 144]
input_shape = x_content.shape[1:]
print(input_shape)
#model = dnn_classifier(input_shape, num_classes)
model = dnn_gap_classifier(input_shape, num_classes)


In [ ]:

initial_epoch = 0


In [ ]:


def loadFeatureVector(file_path):
    return np.load(file_path, allow_pickle=True)
    
def featureVectorLoader(data_split, data_type):
    #every file has 35 feature vectors (one batch)   
    x_images = data_split.get(data_type, None)
    while True:
        for participant, sessDict in x_images.items():
            for sess, runDict in sessDict.items():
                for run in runDict.keys():
                    file_path= os.path.join(stimuli_features_dir, "_".join([participant, sess, run]) + ".npy")
                    X = loadFeatureVector(file_path)
                    Y = utils.to_categorical(np.transpose(y_labels[data_type][participant][sess][run]))
                    yield (X,Y)

EPOCHS=20
#callbacks
callbacks = [EarlyStopping(monitor='val_loss', patience=4),
             ModelCheckpoint(filepath='weights.{epoch:02d}.h5', monitor='val_loss', verbose=1)]

#callbacks = [ModelCheckpoint(filepath='weights.{epoch:02d}.h5', monitor='val_loss', verbose=1)]

model.compile(optimizer='adam', loss='categorical_crossentropy', metrics=['categorical_accuracy'])
#train_history = model.fit(x=x_train, y=y_train, epochs=EPOCHS, callbacks=callbacks, batch_size=35, validation_data=(x_test, y_test))
#train_history = model.fit(x=x_train, y=y_train, epochs=EPOCHS, callbacks=callbacks, batch_size=35, validation_data=(x_test, y_test))
#train_history = model.fit_generator(featureVectorLoader(x_images_path), steps_per_epoch=5, epochs=EPOCHS, callbacks=callbacks) 
#train_history = model.fit_generator(featureVectorLoader(x_images_path), steps_per_epoch=5, epochs=EPOCHS, callbacks=callbacks, validation_data=(x_test, y_test)) 

#steps_per_epoch = (last_sess - 1) * (last_run - 1)

numberOfSessions = data_split["train"]["last_sess"] - data_split["train"]["start_sess"]
numberOfRuns = data_split["train"]["last_run"] - data_split["train"]["start_run"]
numberOfParticipants = len(data_split["train"]["participant_list"])
steps_per_epoch = numberOfSessions * numberOfRuns * numberOfParticipants

numberOfSessions = data_split["dev"]["last_sess"] - data_split["dev"]["start_sess"]
numberOfRuns = data_split["dev"]["last_run"] - data_split["dev"]["start_run"]
numberOfParticipants = len(data_split["dev"]["participant_list"])
validation_steps = numberOfSessions * numberOfRuns * numberOfParticipants

print("Total number of training examples: %s" % (steps_per_epoch * 37))
print("Total number of dev examples: %s" % (validation_steps * 37))

print("steps_per_epoch: %s" % steps_per_epoch)
#train_history = model.fit_generator(featureVectorLoader(x_images_path), steps_per_epoch=350, epochs=EPOCHS, validation_data=(x_test, y_test)) 
#train_history = model.fit_generator(featureVectorLoader(x_images_path, "train"), steps_per_epoch=350, epochs=EPOCHS, validation_data=featureVectorLoader(x_images_path, "train"), validation_steps=350) 
train_history = model.fit_generator(featureVectorLoader(x_images_path, "train"), steps_per_epoch=steps_per_epoch, epochs=EPOCHS,
                                    callbacks=callbacks, validation_data=featureVectorLoader(x_images_path, "dev"),
                                    validation_steps=validation_steps) 
#train_history = model.fit_generator(featureVectorLoader(x_images_path, "train"), steps_per_epoch=steps_per_epoch, epochs=EPOCHS,
#                                    callbacks=callbacks, validation_data=(x_dev, y_dev))


loss = train_history.history['loss']
val_loss = train_history.history['val_loss']
plt.plot(loss)
plt.plot(val_loss)
plt.legend(['loss', 'val_loss'])
plt.show()



In [ ]:
# load model
new_model = load_model('weights.20.h5')
new_model.summary()
print(new_model.get_weights()[0].shape)
print(new_model.get_weights()[1].shape)
print(new_model.get_weights()[2].shape)
print(new_model.get_weights()[3].shape)


In [ ]:

N=50
arr1 = new_model.get_weights()[2][:,0]
indices1 = np.argsort(arr1, axis=0)[-N:]
arr2 = new_model.get_weights()[2][:,1]
indices2 = np.argsort(arr2, axis=0)[-N:]
arr3 = new_model.get_weights()[2][:,2]
indices3 = np.argsort(arr3, axis=0)[-N:]
#
print(indices1)
print(indices2)
print(indices3)

In [ ]:
from collections import Counter
arr = new_model.get_weights()[0]
N=20
filter_select = []
for index_list in [indices1, indices2, indices3]:
    all_ind = []
    for index in index_list:
        indices = np.argsort(arr, axis=0)[-N:, index]
        sort_ind = np.sort(indices, axis=-1)
        all_ind.extend(list(sort_ind))
        #print(sort_ind)
        #plt.plot(sort_ind)
    a_ind = [key for key,_ in Counter(all_ind).most_common()][0:10]
    print(a_ind)
    filter_select.extend([item for item in a_ind if item not in filter_select])
    
print(filter_select)


In [ ]:
#!pip install nibabel
import nibabel as nib
import re
fmri_data_dir = '/home/ubuntu/cs230Project/dataset/ds001499-download'
stimuli_features_dir = 'stimulifeatures'
fmriRegex = re.compile(r'^(.*?)_sess(.*?)_run(.*?).npy$')

# At the beginning and end of each run, a fixation cross was shown for 6 sec (3TORs) and
# 12 sec (6TORs), respectively. hence stIndex goes from 3:-6
# 37 images shows in each run >> 185 TOR
# Each image was presented for 1 sec followed by a 9 sec fixation cross (5TORs)
# For each stimuls, average assocated 5 TORs and map them
def loadFmriData(file_path):
    x_train = []
    epi_img = nib.load(file_path)
    img_data = epi_img.get_fdata()
    for stIndex in range(4,  img_data.shape[-1] - 5, 5):
        x_train.append(np.mean(img_data[:,:,:,stIndex:stIndex+5], axis=-1))

    x = np.asarray(x_train)
    return x

def loadFilterVector(file_path, filterNumList):
    all_features = np.load(file_path, allow_pickle=True)
    features = []
    for filterNum in filterNumList:
        features.append(all_features[:, filterNum, :].T)
    
    ft = np.asarray(features)
    return ft.reshape(-1, 37).T

filterNumList = [452, 209, 327, 377, 33, 16, 433, 19, 66, 467]
data_split = x_images_path
data_type = "train"
x_images = data_split.get(data_type, None)
for participant, sessDict in x_images.items():
    for sess, runDict in sessDict.items():
        for run in runDict.keys():
            #fmri_data_path = os.path.join(fmri_data_dir, "sub-%s" % participant, "sess" "_".join([participant, sess, run]) + ".npy")
            feature_file_name = "_".join([participant, sess, run]) + ".npy"
            #sub-CSI3/ses-01/func
            # sub-CSI3_ses-09_task-5000scenes_run-05_bold.nii.gz
            match = fmriRegex.match(feature_file_name)
            if match:
                  fmri_file_name = "sub-%s_ses-%s_task-5000scenes_run-%s_bold.nii.gz" % ( match.group(1), match.group(2), match.group(3))
                  fmri_data_path = os.path.join(fmri_data_dir, "sub-%s" % match.group(1), "ses-%s" % match.group(2), "func", fmri_file_name)
                  print(fmri_data_path)
                
            feature_vector_path= os.path.join(stimuli_features_dir, feature_file_name)
            X = loadFmriData(fmri_data_path)
            Y = loadFilterVector(feature_vector_path, filterNumList)
            print(X.shape)
            print(Y.shape)
            break

                                                      

In [17]:
# Train a single layer neural network (one network per filter) to map fmri data to above filters
# input: X of shape (106, 106, 69, 194) 
# output: Y of shape (144, 1) image features on specific filters
fmriRegex = re.compile(r'^(.*?)_sess(.*?)_run(.*?).npy$')

fmri_data_dir = '/home/ubuntu/cs230Project/dataset/ds001499-download'
stimuli_features_dir = 'stimulifeatures'

def loadFilterVector(file_path, filterNumList):
    all_features = np.load(file_path, allow_pickle=True)
    #features = []
    #for filterNum in filterNumList:
    #    features.append(all_features[:, filterNum, :].T)
    
    #ft = np.asarray(features)
    #return ft.reshape(-1, 37).T
    #return all_features[:, 377, :]
    feat_sel = all_features[:, filterNumList, :]
    sel_shape = feat_sel.shape[0]
    return feat_sel.reshape(sel_shape, -1)

# At the beginning and end of each run, a fixation cross was shown for 6 sec (3TORs) and
# 12 sec (6TORs), respectively. hence stIndex goes from 3:-6
# 37 images shows in each run >> 185 TOR
# Each image was presented for 1 sec followed by a 9 sec fixation cross (5TORs)
# For each stimuls, average assocated 5 TORs and map them
def loadFmriData(file_path):
    x_train = []
    epi_img = nib.load(file_path)
    img_data = epi_img.get_fdata()
    for stIndex in range(4, img_data.shape[-1] - 5, 5):
        #x_train.append(np.mean(img_data[:,:,:,stIndex:stIndex+5], axis=-1))
        x_train.append((img_data[:,:,:,stIndex+3]))

    x = np.asarray(x_train)
    return x

def featureVectorLoader(data_split, data_type, filterNum):
    #every file has 35 feature vectors (one batch)
    L = len(fileList)   
    x_images = data_split.get(data_type, None)
    while True:
        for participant, sessDict in x_images.items():
            for sess, runDict in sessDict.items():
                for run in runDict.keys():
                    #fmri_data_path = os.path.join(fmri_data_dir, "sub-%s" % participant, "sess" "_".join([participant, sess, run]) + ".npy")
                    feature_file_name = "_".join([participant, sess, run]) + ".npy"
                    #sub-CSI3/ses-01/func
                    # sub-CSI3_ses-09_task-5000scenes_run-05_bold.nii.gz
                    match = fmriRegex.match(feature_file_name)
                    if match:
                        fmri_file_name = "sub-%s_ses-%s_task-5000scenes_run-%s_bold.nii.gz" % ( match.group(1), match.group(2), match.group(3))
                        fmri_data_path = os.path.join(fmri_data_dir, "sub-%s" % match.group(1), "ses-%s" % match.group(2), "func", fmri_file_name)
                
                    feature_vector_path= os.path.join(stimuli_features_dir, feature_file_name)
                    X = loadFmriData(fmri_data_path)
                    Y = loadFilterVector(feature_vector_path, filterNum)
                    yield (X,Y)


# for each y, we have 
def auto_encoder(input_shape, encoding_dim):
    X_input = Input(input_shape)
    #X = Conv2D(2, (5,5), activation='relu')(X_input)
    #X = MaxPooling2D(pool_size=(2, 2))(X)
    #X = Dropout(0.25)(X)
    X = Conv2D(4, (1,1), activation='tanh')(X_input)
    X = Flatten()(X)
    X = Dense(64, activation='relu')(X)
    X = Dropout(0.4)(X)
    X = Dense(encoding_dim, activation='relu')(X)
    model = Model(inputs = X_input, outputs = X, name='auto_encoder')
    return model

EPOCHS = 100
filterNumList = [452, 209, 327, 377, 33, 16, 433, 19, 66, 467]
#filterNumList = [377]
numberOfSessions = data_split["train"]["last_sess"] - data_split["train"]["start_sess"]
numberOfRuns = data_split["train"]["last_run"] - data_split["train"]["start_run"]
numberOfParticipants = len(data_split["train"]["participant_list"])
steps_per_epoch = numberOfSessions * numberOfRuns * numberOfParticipants

numberOfSessions = data_split["dev"]["last_sess"] - data_split["dev"]["start_sess"]
numberOfRuns = data_split["dev"]["last_run"] - data_split["dev"]["start_run"]
numberOfParticipants = len(data_split["dev"]["participant_list"])
validation_steps = numberOfSessions * numberOfRuns * numberOfParticipants

encoder_model = auto_encoder((106,106,69), len(filterNumList)*144)
#cosine_proximity
encoder_model.compile(optimizer='adam', loss='mean_squared_error', metrics=['accuracy'])
train_history = encoder_model.fit_generator(featureVectorLoader(x_images_path, "train", filterNumList), steps_per_epoch=steps_per_epoch, epochs=EPOCHS,
                                    validation_data=featureVectorLoader(x_images_path, "dev", filterNumList),
                                    validation_steps=validation_steps) 

Epoch 1/100


InternalError: 2 root error(s) found.
  (0) Internal: Dst tensor is not initialized.
	 [[{{node _arg_input_14_0_0}}]]
	 [[loss_11/mul/_945]]
  (1) Internal: Dst tensor is not initialized.
	 [[{{node _arg_input_14_0_0}}]]
0 successful operations.
0 derived errors ignored.

# Predict

In [ ]:

x = unrollContentOutput(predictContentOut)
x = np.expand_dims(x, axis=0)
print('Input image shape:', x.shape)
print(model.predict(x))

model2.compile(optimizer='adam', loss='categorical_crossentropy', metrics=['accuracy'])


# Load processed fmri data

In [ ]:
num_classes = 4

def dnn_classifier(input_shape, num_classes):
    X_input = Input(input_shape)
    X = Flatten()(X_input)
    X = Dense(512, activation='tanh')(X)
    X = Dense(128, activation='tanh')(X)
    X = Dense(num_classes, activation='softmax')(X)
    model = Model(inputs = X_input, outputs = X, name='dnn_classifier')
    return model

input_shape = x_content.shape[1:]
model2 = dnn_classifier(input_shape, num_classes)

EPOCHS=100
#callbacks
#callbacks = [EarlyStopping(monitor='val_loss', patience=2),
#             ModelCheckpoint(filepath='weights.{epoch:02d}.h5', monitor='val_loss', verbose=1)]

callbacks = [ModelCheckpoint(filepath='weights.{epoch:02d}.h5', monitor='val_loss', verbose=1)]

model2.compile(optimizer='adam', loss='categorical_crossentropy', metrics=['accuracy'])
#train_history = model.fit(x=x_train, y=y_train, epochs=EPOCHS, callbacks=callbacks, batch_size=35, validation_data=(x_test, y_test))
#train_history = model.fit(x=x_train, y=y_train, epochs=EPOCHS, callbacks=callbacks, batch_size=35, validation_data=(x_test, y_test))
#train_history = model.fit_generator(featureVectorLoader(x_images_path), steps_per_epoch=5, epochs=EPOCHS, callbacks=callbacks) 
#train_history = model.fit_generator(featureVectorLoader(x_images_path), steps_per_epoch=5, epochs=EPOCHS, callbacks=callbacks, validation_data=(x_test, y_test)) 

#steps_per_epoch = (last_sess - 1) * (last_run - 1)

numberOfSessions = data_split["train"]["last_sess"] - data_split["train"]["start_sess"]
numberOfRuns = data_split["train"]["last_run"] - data_split["train"]["start_run"]
numberOfParticipants = len(data_split["train"]["participant_list"])
steps_per_epoch = numberOfSessions * numberOfRuns * numberOfParticipants

numberOfSessions = data_split["dev"]["last_sess"] - data_split["dev"]["start_sess"]
numberOfRuns = data_split["dev"]["last_run"] - data_split["dev"]["start_run"]
numberOfParticipants = len(data_split["dev"]["participant_list"])
validation_steps = numberOfSessions * numberOfRuns * numberOfParticipants


print("Total number of training examples: %s" % (steps_per_epoch * 37))
print("Total number of dev examples: %s" % (validation_steps * 37))

print("steps_per_epoch: %s" % steps_per_epoch)
#train_history = model.fit_generator(featureVectorLoader(x_images_path), steps_per_epoch=350, epochs=EPOCHS, validation_data=(x_test, y_test)) 
#train_history = model.fit_generator(featureVectorLoader(x_images_path, "train"), steps_per_epoch=350, epochs=EPOCHS, validation_data=featureVectorLoader(x_images_path, "train"), validation_steps=350) 
#train_history = model.fit_generator(featureVectorLoader(x_images_path, "train"), steps_per_epoch=steps_per_epoch, epochs=EPOCHS,
#                                    callbacks=callbacks, validation_data=featureVectorLoader(x_images_path, "dev"),
#                                    validation_steps=validation_steps) 
train_history = model2.fit(x=x_train, y=y_train, epochs=EPOCHS, callbacks=callbacks, batch_size=35, validation_data=(x_test, y_test))





loss = train_history.history['loss']
val_loss = train_history.history['val_loss']
plt.plot(loss)
plt.plot(val_loss)
plt.legend(['loss', 'val_loss'])
plt.show()
